In [16]:
import numpy as np
import pandas as pd
import os
import skimage
from skimage import io
from skimage import img_as_float
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.metrics import roc_auc_score, f1_score
from os import listdir
from skimage import io, transform, util, img_as_float
from tqdm import tqdm_notebook
from keras_tqdm import TQDMNotebookCallback
from PIL import Image
from fastnumbers import fast_real
from keras import Sequential
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
%matplotlib inline

In [17]:
from tqdm import tqdm

In [18]:
def txt2mask (path):
    yLinesCoords = []
    f = open(path, 'r', encoding='utf-16')
    line = f.readline()
    while line:
        line = f.readline()
        if line.split("	")[0] != "" :
            num1 = line.split("	")[0]
            num2 = line.split("	")[1].split("\n")[0]
            yLinesCoords.append(fast_real(num1))
            yLinesCoords.append(fast_real(num2))
    f.close()
    
    res = np.zeros((h, w))
    i = 0
    while i < len(yLinesCoords)/2:
        j = yLinesCoords[2*i]
        while j < yLinesCoords[2*i + 1]:
            for k in range(w):
                res[j][k] = 1
            j += 1
        i += 1   
    return res

In [19]:
def shape32 (img):
    padded_img = np.zeros((3520, 2496))
    padded_img[:img.shape[0], :img.shape[1]] = img
    return padded_img

In [20]:
n_train_samples = 100
n_test_samples = 20
n_labels = 2
kernel = 15
h = 3520 #3508
w = 2496 #2480

In [10]:
#_train
#X_train = np.zeros((n_train_samples, h, w))
#Y_train = np.zeros((n_train_samples, h, w))

for i, filename in tqdm(enumerate(os.listdir('train_img/')[:n_train_samples])):
    onlyName = filename.split('.')[0]
    img = skimage.io.imread('train_img/' + filename)
    if img.shape != (3508, 2480):
        raise ValueError('yobik')
    shaped = shape32(img)
    normalized = shaped / 255.0
    x_arr = normalized
    y_arr = txt2mask('train_txt/' + onlyName + '.txt')
    np.savez_compressed('saved/train/' + onlyName + '.npz', x=x_arr, y=y_arr)
    #X_train[i] = normalized
    #Y_train[i] = txt2mask('train_txt/' + filename.split('.')[0] + '.txt')
    
    

3it [00:08,  2.82s/it]/home/ivan/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
80it [03:38,  2.73s/it]


In [13]:
#_test
#X_test = np.zeros((n_test_samples, h, w))
#Y_test = np.zeros((n_test_samples, h, w))

for i, filename in tqdm(enumerate(os.listdir('test_img/')[:n_test_samples])):
    img = skimage.io.imread('test_img/' + filename)
    if img.shape != (3508, 2480):
        raise ValueError('yobik')
    shaped = shape32(img)
    normalized = shaped / 255.0
    x_arr = normalized
    y_arr = txt2mask('test_txt/' + filename.split('.')[0] + '.txt')
    np.savez_compressed('saved/test/' + filename.split('.')[0] + '.npz', x=x_arr, y=y_arr)
    #X_test[i] = normalized
    #Y_test[i] = txt2mask('test_txt/' + filename.split('.')[0] + '.txt')
    

3it [00:08,  2.83s/it]/home/ivan/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
20it [00:56,  2.82s/it]


In [ ]:
np.savez_compressed('saved/arrays_train.npz', xtrain=X_train, ytrain = Y_train)
np.savez_compressed('saved/arrays_test.npz', xtest=X_test, ytest = Y_test)

In [44]:
def img_generator(directory, batch_size):
    filenames = os.listdir(directory)
    while True:
        n_iter = len(filenames) // batch_size
        residual = len(filenames) % batch_size
        for i in range(n_iter):
            size = batch_size
            if (i == n_iter - 1) and (residual > 0):
                size = residual
            X = np.zeros((batch_size, h, w, 1))
            y = np.zeros((batch_size, h * w))
            for j in range(size):
                X[j] = np.load(directory + filenames[i * batch_size + j])['x'].reshape((h, w, 1))
                y = np.load(directory + filenames[i * batch_size + j])['y'].reshape((h * w, ))
                y[j] = 
            yield X, y

In [37]:
train_generator = img_generator('saved/train/', 5)
test_generator = img_generator('saved/test/', 5)

In [ ]:
loaded_train = np.load('saved/arrays_train.npz')
loaded_test = np.load('saved/arrays_test.npz')
X_train = loaded_train['xtrain']
Y_train = loaded_train['ytrain']
X_test = loaded_test['xtest']
Y_test = loaded_test['ytest']

In [38]:
autoencoder = Sequential()
encoding_layers = [
    Convolution2D(16, kernel, kernel, border_mode='same', input_shape=( 3520, 2496, 1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
#    
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
#    
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
#
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
#   
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
#    
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
]


for l in encoding_layers:
    autoencoder.add(l)
    #print(l.input_shape,l.output_shape,l)

decoding_layers = [
    UpSampling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    
#   
    UpSampling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    
#   
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
#
    UpSampling2D(),
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
#   
    UpSampling2D(),
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
#
    UpSampling2D(),
    Convolution2D(16, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(n_labels, 1, 1, border_mode='valid'),
    BatchNormalization(),
    
]

for l in decoding_layers:
    autoencoder.add(l)
    #print(l.input_shape,l.output_shape,l)
autoencoder.add(Reshape((n_labels, 3520 * 2496)))
autoencoder.add(Permute((2, 1)))
autoencoder.add(Activation('softmax'))

/home/ivan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (15, 15), input_shape=(3520, 249..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ivan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (15, 15), padding="same")`
  
/home/ivan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (15, 15), padding="same")`
  del sys.path[0]
/home/ivan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (15, 15), padding="same")`
/home/ivan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (15, 15), padding="same")`
/home/i

In [39]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 3520, 2496, 16)    3616      
_________________________________________________________________
batch_normalization_20 (Batc (None, 3520, 2496, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 3520, 2496, 16)    0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1760, 1248, 16)    0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 1760, 1248, 32)    115232    
_________________________________________________________________
batch_normalization_21 (Batc (None, 1760, 1248, 32)    128       
_________________________________________________________________
activation_21 (Activation)   (None, 1760, 1248, 32)    0         
__________

In [45]:
train_generator = img_generator('saved/train/', 5)
test_generator = img_generator('saved/test/', 5)

autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
autoencoder.fit_generator(train_generator, steps_per_epoch=100//5, epochs=10, validation_data=test_generator, validation_steps=4)

Epoch 1/10


ValueError: Error when checking target: expected activation_32 to have shape (8785920, 2) but got array with shape (3520, 2496)

In [ ]:
def getLinesFromFile (path):
    data = []
    f = open(path, 'r', encoding='utf-16')
    line = f.readline()
    while line:
        line = f.readline()
        if line.split("	")[0] != "" :
            num1 = line.split("	")[0]
            num2 = line.split("	")[1].split("\n")[0]
            data.append(fast_real(num1))
            data.append(fast_real(num2))
    f.close()
    return data

In [ ]:
def getTextAndBackAreas (yLinesCoords):
    res = np.zeros(3508)
    i = 0
    while i < len(yLinesCoords)/2:
        j = yLinesCoords[2*i]
        while j < yLinesCoords[2*i + 1]:
            res[j] = 1
            j += 1
        i += 1   
    return res

In [ ]:
def convertJpg2Png (path):
    im = Image.open(path)
    im.save(path.split('.')[0] + '.png')
    pass

In [ ]:
def convert2greyscale (path):
    img = Image.open(path).convert('LA')
    img.save(path.split('.')[0] + '_greyscale.png')
    pass

In [ ]:
def resizePng (path, newWidth, newHeigth):
    img = Image.open(path)
    resized_img = img.resize((newWidth, newHeigth), Image.ANTIALIAS)
    resized_img.save(path.split('.')[0] + '_resized.png')
    pass

In [ ]:
def agregate2array (path, width, heigth):
    img = Image.open(path)
    pix = np.array(img)
    res = []
    for i in range(heigth):
        greySum = 0
        for j in range(width):
            greySum += pix[i][j][0]/pix[i][j][1]
        greyAverage = greySum/width
        res.append(greyAverage)
    return np.array(res)

In [ ]:
path_to_lines = "582.txt"
path_to_img = "582.jpg"
width = 2480
heigth = 3508

In [ ]:
yLinesCoords = getLinesFromFile (path_to_lines)
oneZeroArr = getTextAndBackAreas(yLinesCoords)

convertJpg2Png (path_to_img)
convert2greyscale (path_to_img)
greyAgrArr = agregate2array (path_to_img.split('.')[0] + '_greyscale.png', width, heigth)

In [ ]:
print (oneZeroArr.shape)

In [ ]:
print (greyAgrArr.shape)

In [ ]:
oneZeroArr = oneZeroArr.reshape (1, -1)
greyAgrArr = greyAgrArr.reshape (1, -1)

In [ ]:
print (oneZeroArr.shape)
print (greyAgrArr.shape)

In [ ]:
path_to_img2 = "583.jpg"
convertJpg2Png (path_to_img2)
convert2greyscale (path_to_img2)
greyAgrArr2 = agregate2array (path_to_img2.split('.')[0] + '_greyscale.png', width, heigth)
greyAgrArr2 = greyAgrArr2.reshape(1, -1)

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Activation

max_features = 1000
maxlen = 3508 
batch_size = 1

model = Sequential()
model.add(Embedding(max_features, 3508, input_length=maxlen))
model.add(LSTM(3508))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam'
              )

model.fit(
    greyAgrArr, oneZeroArr, 
    batch_size=batch_size, 
    nb_epoch=1,
    verbose = 1,
    
)

result = model.predict_proba(greyAgrArr2)
print(result)

In [ ]:
result2 = model.predict_proba(greyAgrArr2)
print(result)

In [ ]:
yLinesCoords2 = getLinesFromFile ('583.txt')
oneZeroArr2 = getTextAndBackAreas(yLinesCoords2)

In [ ]:
roc_auc_score(oneZeroArr2, result[0])

In [ ]:
f1_score(result[0] > 0.5, oneZeroArr2)

In [ ]:
model.save_weights("saved_best_model.h5")